In [110]:
import matplotlib
import mne
from scipy.signal import stft

import numpy

%matplotlib inline

In [111]:
epochs = mne.read_epochs('files/filtered_epo.fif')
data = epochs.get_data()
data.shape

Reading files/filtered_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    4816.00 ms
        0 CTF compensation matrices available
125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


(125, 16, 1205)

In [112]:
# aplicando STFT
_, _, w = stft(data, fs=241, nperseg=32, noverlap=16)
print(w.shape)

(125, 16, 17, 77)


In [113]:
def shape_characteristic(array):
    array = array.transpose(0, 2, 1)
    return numpy.reshape(array, (array.shape[0] * array.shape[1], array.shape[2]))

## Média

In [114]:
result = numpy.mean(w, -1)
mean = shape_characteristic(result)
mean.shape

(2125, 16)

## Maximo

In [115]:
result = numpy.max(w, -1)
_max = shape_characteristic(result)
_max.shape

(2125, 16)

## Minimo

In [116]:
result = numpy.min(w, -1)
_min = shape_characteristic(result)
_min.shape

(2125, 16)

## Desvio padrão

In [117]:
result = numpy.std(w, -1)
std = shape_characteristic(result)
std.shape

(2125, 16)

# PSD

In [118]:
w_psd = numpy.abs(w) ** 2
print(w.shape)

(125, 16, 17, 77)


## Média do PSD

In [119]:
result = numpy.mean(w_psd, -1)
mean_psd = shape_characteristic(result)
mean_psd.shape

(2125, 16)

## Desvio padrão do PSD

In [120]:
result = numpy.std(w_psd, -1)
std_psd = shape_characteristic(result)
std_psd.shape

(2125, 16)

## Concatenando tudo

In [121]:
numpy.concatenate((mean, _max, _min, std, mean_psd, std_psd)).shape

(12750, 16)